In [2]:
# 安装pdfminer
!pip install pdfminer.six

### 1. pdf页面读取时，保证读取顺序

In [ ]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBox, LTTextLine, LTChar, LTImage


# 读取 PDF 文件
file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

print(os.getcwd())
# 获取当前目录
current_directory = os.getcwd()
# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))

file_path = os.path.join(grandparent_directory, file_directory)
print(file_path)




# 定义一个函数来提取并排序文本
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
        print(sorted_elements)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text().replace(" ", "")
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text


#提取并排序文本
all_text = extract_and_sort_text(file_path)

# 打印每一页的文本
for i, text in enumerate(all_text):
    print(f"Page {i + 1}:")
    print(text)
    print("\n")


##### 页面元素排序原理解释
```python
def extract_and_sort_text(file_path):
    all_text = []

    # 提取每一页的布局信息
    for page_layout in extract_pages(file_path):
        # 按位置排序布局元素
        print(page_layout)
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)

        # 合并文本
        current_text = ""
        for element in sorted_elements:
            if isinstance(element, LTTextBox) or isinstance(element, LTTextLine):
                current_text += element.get_text()
            elif isinstance(element, LTImage):
                print("Image at position:", element.bbox)

        all_text.append(current_text.strip())

    return all_text
```

`sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)` 

这段代码用来保证读取的pdf文档内容按顺序呈现，通过这种方式排列布局元素的原因有以下几点：
1. 在 pdfminer 中，元素坐标系统如下：
    - 原点 (0, 0)：位于页面的左下角。
    - x 轴：从左向右增加，一直到X轴边界。
    - y 轴：从下向上增加，一直到Y轴边界。
2. 排序目标
    - 首先按照 y1大小顺序， 将元素在Y轴上从上到下排序。reverse=True，即在Y轴方向上从PDF的上边界往Y轴的原点方向排列。
    - 当不同元素的y1相同时，则按x0从小到大顺序排列，即在X轴方向从PDF的X轴初始方向往右边界排列。
    - 经过以上在X，Y轴方向的排序步骤之后，sorted_elements 列表中存储的元素顺序符合人类查看PDF的习惯，从上到下，从左到右。


In [ ]:
# import os
# from pdfminer.high_level import extract_pages
# from pdfminer.layout import LTTextLine, LTTextBox,LAParams
# from collections import Counter

# # 读取 PDF 文件
# file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

# # 获取当前目录
# current_directory = os.getcwd()
# print(f"当前目录: {current_directory}")

# # 获取当前目录的上上层目录
# grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))
# print(f"上上层目录: {grandparent_directory}")

# # 拼接文件路径
# file_path = os.path.join(grandparent_directory, file_directory)
# print(f"文件路径: {file_path}")

# def identify_first_line_indent(page_layout):
#     """
#     识别页面中的首行缩进。
#     """
#     # 筛选文本行
#     lines = [obj for obj in page_layout if isinstance(obj, LTTextBox) or isinstance(obj, LTTextLine)]
#     print(f"文本行数量: {len(lines)}")
    
#     # 按 y 坐标排序文本行
#     lines.sort(key=lambda line: line.bbox[1], reverse=True)
    
#     indent_results = []
    
#     # 遍历每一行文本
#     for i, line in enumerate(lines):
#         # if i == 0:
#         #     # 第一行不计算缩进
#         #     continue
        
#         prev_line = lines[i-1]
        
#         # 计算当前行和前一行的 x 起始位置
#         current_x_start = line.bbox[0]
#         # prev_x_start = prev_line.bbox[0]
#         # # 每页面第一行 x 起始位置
#         # prev_x_start = lines[0].bbox[0]
#         # 每页面 x轴的起始位置0
#         prev_x_start = 0
        
#         # 计算首行缩进
#         indent = current_x_start - prev_x_start
        
#         indent_results.append((line.get_text().strip(), indent))
    
#     return indent_results


# # 统计页面元素中文本行各种缩进值的个数，取个数最多且缩进值最大缩进值，且不为0
# def filter_max_indent(indents):
#     indent_counts = Counter(indent for _, indent in indents)
#     # for i, count in indent_counts.items():
#     #     print(f"首行缩进: {i:.2f} 像素, 出现次数: {count}")
#     # 将统计出的结果dict，按出现的次数降序排序，即按value排序，item[1]是value
#     sorted_itmes_by_value = sorted(indent_counts.items(), key = lambda item:item[1], reverse=True)

#     # 取出现频次最高的前三组键值对（已转成tuple）
#     max3_items_by_value = sorted_itmes_by_value[:3]

#     # 对出现频次最高的前三对键值对（已转成tuple），再按第一个元素降序排序，item[0]是第一个元素
#     sorted_itmes_by_key = sorted(max3_items_by_value,key = lambda item:item[0],reverse=True)

#     # 取第一个元素最大的tuple对的第一个元素
#     indent = sorted_itmes_by_key[0]
#     print(f'当前页的缩进值是：{indent}')
    

# def filter_max_indent2(indents):
#     dict1 = Counter(indent for _, indent in indents)
#     indent_counts = list(dict1.items())
#     # tuples_list = [(237.36,1),(318.24,1),(95.16,3),(343.44,1),(305.88,1),(95.40,1)]

#     # 目标差值范围
#     min_diff = 29.00
#     max_diff = min_diff + 4.00

#     # 存储符合条件的tuple组合
#     valid_pairs = []

#     # 遍历所有可能的tuple组合
#     for i in range(len(indent_counts)):
#         for j in range(i + 1, len(indent_counts)):
#             diff = abs(indent_counts[i][0] - indent_counts[j][0])
#             if min_diff <= diff <= max_diff:
#                 valid_pairs.append((indent_counts[i][0], indent_counts[j][0]))
#                 max_value = max(max(pair) for pair in valid_pairs) if len(valid_pairs)>0 else 0
#                 return max_value
#                 # max_value = max(max(pair) for pair in valid_pairs)
#                 print(f"当前页的缩进值是: {max_value}")

# # 读取 PDF 页面
# for page_layout in extract_pages(file_path, laparams=LAParams()):
#     print(f"页面布局对象: {page_layout}")
    

#     # indents = identify_first_line_indent(page_layout)
#     sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
#     indents = identify_first_line_indent(sorted_elements)
#     print(f"首行缩进结果数量: {len(indents)}")
    
    
#     page_indent_value = filter_max_indent2(indents)
#     result_text=[]
#     if len(indents) == 0:
#         print("没有检测到首行缩进。")
#     else:
#         for index, (text, indent) in enumerate(indents):
#             # if indents[index][1] - indents[index-1][1] > 0:
#             #     text = f"\t{text}" 
#             # filter_max_indent2(indents)
#             # indent_counts = Counter(indent for _, indent in indents)
#             # for i, count in indent_counts.items():
#             #     print(f"首行缩进: {i:.2f} 像素, 出现次数: {count}")
#             # !!!!!!!!!!!!
#             if page_indent_value is not None and page_indent_value >0:
#                 if abs(page_indent_value - indent) == 0 or abs(page_indent_value - indent) >=1 :
#                     # text = f"\t{text}"
#                     # print(f"文本: '{text}', 首行缩进: {indent:.2f} 像素")
#                     result_text.append(f"\t{text}")
#                 else:
#                     result_text.append(text)
#             else:
#                 result_text.append(text)
#                 # print(f"处理后的文本:{result_text}")
#     print(result_text)
    
#     # 打印每行文本的详细信息
#     lines = [obj for obj in page_layout if isinstance(obj, LTTextLine)]
#     for line in lines:
#         print(f"文本: '{line.get_text().strip()}', bbox: {line.bbox}")

~~ ### 找出文中文档中每页的段落结构 ~~ 
~~方案一、使用collections库统计出页面上所有文本行首行缩进的像素值，即各个像素值出现的次数。然后取出现次数最多的像素值（前三），再然后取缩进像素值最大的元素，以还原经过处理后的文本原有的段落结构~~

In [ ]:
# from collections import Counter

# abc = [{'a': 3, 'b': 8, 'c': 1, 'd': 12,'e':12,'f':12,'g':1,'h':12,'i':3,'j':99,'k':99,'l':99}]

# for k,v in abc[0].items():
#     print(k,v)

# # 统计字典中各种键的值出现的次数，这里统计的是value即v出现的次数    
# cde = Counter(v for k,v in abc[0].items())
# print(f'统计字典中值出现的次数 {cde}')

# # 将统计出的结果dict，按出现的次数降序排序，即按value排序，item[1]是value
# sorted_itmes_by_value = sorted(cde.items(), key = lambda item:item[1], reverse=True)

# # 取出现频次最高的前三组键值对（已转成tuple）
# max3_items_by_value = sorted_itmes_by_value[:3]

# # 对出现频次最高的前三对键值对（已转成tuple），再按第一个元素降序排序，item[0]是第一个元素
# sorted_itmes_by_key = sorted(max3_items_by_value,key = lambda item:item[0],reverse=True)

# # 取第一个元素最大的tuple对的第一个元素
# indent = sorted_itmes_by_key[0]
# print(f'按统计结果字典键值对的value降序排序结果，{sorted_itmes_by_value}')
# print(f'按第一次排序结果列表中的第一个元素再次降序排序，{sorted_itmes_by_key}')
# print(f'取列表中元组第一个元素最大的元组的第一个元素{indent[0]}')

~~### 找出文中文档中每页的段落结构~~
~~#### 方案二~~
~~    1、使用collections库统计出页面上所有文本行首行缩进的像素值，即各个像素值出现的次数。然后取出现次数最多的像素值（前3个），这里3可以作为变量参数，也可以根据实际页面情况调整数值。~~
~~    2、在统计结果中，找出首行缩进像素值相差27像素到33像素之间（这里27-33可以根据实际情况调整,左右误差3像素）~~
~~    3、在最终的两组结果中，找出缩进值最大的像素值。~~
~~    4、遍历每页，判断每行行首缩进是否接近这个最大像素值（左右可以偏差3像素，视实际情况调整），如果接近则认为该行的首部需要缩进，且是段落开始位置。~~


In [ ]:
# from collections import Counter

# # 每页中各种缩进出现的次数(缩进值,出现的次数)
# page1 = [(190.44,1),(104.1,1),(79.44,5),(109.44,8),(470.64,1)]
# page2 = [(109.44,4),(79.44,12),(95.40,1),(95.40,1)]
# page3 = [(109.44,10),(79.44,13),(111.00,2),(470.64,1)]
# page4 = [(79.44,14),(111.00,8),(95.40,1)]
# page5 = [(79.44,17),(111.00,5),(158.40,2),(470.64,1)]
# page6 = [(237.36,1),(318.24,1),(95.16,3),(343.44,1),(305.88,1),(95.40,1)]
# current_page = page1

# for k,v in current_page:
#     print(k,v)

# # 统计字典中各种键的值出现的次数，这里统计的是value即v出现的次数    
# # current_array = Counter(v for k,v in current_page)

# # # indent_counts = Counter(indent for _, indent in indents)

# # print(f'统计字典中值出现的次数 {current_array}')

# # 将统计出的结果list，按出现的次数降序排序，即按value排序，item[1]是value
# sorted_itmes_by_value = sorted(current_page, key = lambda item:item[1], reverse=True)
# print(f'按统计结果字典键值对的value降序排序结果，{sorted_itmes_by_value}')

# # 取出现频次最高的前三组键值对（已转成tuple）
# max3_items_by_value = sorted_itmes_by_value[:3]
# print(f'按第一次排序结果列表中的第一个元素再次降序排序，{sorted_itmes_by_key}')

# # # 对出现频次最高的前三对键值对（已转成tuple），再按第一个元素降序排序，item[0]是第一个元素
# # sorted_itmes_by_key = sorted(max3_items_by_value,key = lambda item:item[0],reverse=True)

# # 取第一个元素最大的tuple对的第一个元素
# # indent = sorted_itmes_by_key[0]
# # print(f'取列表中元组第一个元素最大的元组的第一个元素{indent[0]}')

In [ ]:
# tuples_list = [(237.36,1),(318.24,1),(95.16,3),(343.44,1),(305.88,1),(95.40,1)]

# # 目标差值范围
# min_diff = 29.00
# max_diff = min_diff + 4.00

# # 存储符合条件的tuple组合
# valid_pairs = []

# max_value = 0
# # 遍历所有可能的tuple组合
# for i in range(len(tuples_list)):
#     for j in range(i + 1, len(tuples_list)):
#         diff = abs(tuples_list[i][0] - tuples_list[j][0])
#         if min_diff <= diff <= max_diff:
#             valid_pairs.append((tuples_list[i][0], tuples_list[j][0]))
#             if len(valid_pairs) > 0:
#                 max_value = max(max(pair) for pair in valid_pairs)
#             else:
#                 max_value = 0
# # 输出结果
# print("符合条件的tuple组合:", valid_pairs)
# print("最大值:", max_value)

### 2. PDF文件分段策略
1. 找出当前页的，所有LTTextBoxHorizontal中的x0最小值，和x1最大值，最小值x0就是段落起始位置。
2. 当前行x0位置大于起始位置且当前行x0大于上一行x0位置就换行。当前行x0位置和上一行x0位置一样，或者当前行等于起始位置就不做处理。

In [79]:
import os
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LAParams
# 读取 PDF 文件
file_directory = "not_upload\\关于开展公众市场三季度营销活动的通知.pdf"

# print(os.getcwd())
# 获取当前目录
current_directory = os.getcwd()
# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))

file_path = os.path.join(grandparent_directory, file_directory)

page_numbers = [0]  # 指定提取第1页和第2页

laparams = LAParams(
    line_overlap=0.5,
    char_margin=2.0,
    line_margin=0.5,
    word_margin=0.1,
    boxes_flow=1.0,
    detect_vertical=False,
    all_texts=False
)

# 页面提取，元素排序还原
def extract_pdf(file_path,page_numbers):
    for idx,page_layout in enumerate(extract_pages(file_path,page_numbers=page_numbers)) :
        sorted_elements = sorted(page_layout, key=lambda x: (x.y1, x.x0), reverse=True)
        element = [element for element in sorted_elements if isinstance(element, LTTextContainer)]
        # 去除页面水印信息
        text_line = [text_line for text_line in element if text_line.x0>0]
        # print(text_line)
        return text_line

# 找出页面的缩进值
def find_current_page_xstart(text_line):
    min_x0 = float('inf')
    max_x1 = float('-inf')

    # 遍历 text_line 列表，更新最小值和最大值
    for line in text_line:
        if line.x0 < min_x0:
            min_x0 = line.x0
        if line.x1 > max_x1:
            max_x1 = line.x1

# 查找满足条件的元素
    matching_x0 = None
    for line in text_line:
        if int(line.x0) == int(min_x0) : # and int(line.x1) == int(max_x1)
            matching_x0 = int(line.x0)
            break

    return matching_x0,min_x0,max_x1

text_line = extract_pdf(file_path,page_numbers)
matching_x0,min_x0,max_x1 = find_current_page_xstart(text_line)

print(f"页面起始位置x0: {matching_x0}")
print(f"所有页面元素x0 的最小值: {min_x0}")
print(f"所有页面元素x1 的最大值: {max_x1}")


# 页面经过处理后的段落
paragraphs = []

def pdf_segment(text_line,min_line_length=1):
    buffer = ''
    prev_x0 = 0
    for idx,line in enumerate(text_line):
        str = line.get_text().replace(' ', '').strip()
        # 当前行的x0坐标大于当前页面的起始位置x0,同时当前行的坐标x0大于上一行的x0坐标则换行
        if int(line.x0) > matching_x0  and int(line.x0) > prev_x0:
            buffer += '\n' + str
        # 当前行的x0坐标等于当前页面的起始位置x0,或者等于上一行的x0坐标则不换行，不添加空格
        elif int(line.x0) == matching_x0 or int(line.x0) == prev_x0:
            buffer += ('' + str)
        else:
            buffer += (' ' + str)
        prev_x0 = int(line.x0)
    
    lines = buffer.split('\n')
    for line in lines:
        if(len(line)>=min_line_length):
            paragraphs.append(line)
    print(paragraphs)
    return paragraphs

pdf_segment(text_line,min_line_length=10)


The PDF <_io.BufferedReader name='c:\\Users\\Administrator\\Documents\\Learn-LLM\\not_upload\\关于开展公众市场三季度营销活动的通知.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


页面起始位置x0: 79
所有页面元素x0 的最小值: 79.44
所有页面元素x1 的最大值: 521.8644500000003
['联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：', '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：', '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9534户；宽带发展1.86万户，宽带净增10625户；129元及以上实收高套发展7360户，FTTR发展4600户，奋力实现三季度收入预算完成率75%。', '三、营销部署（一）市场策略1.发展侧']


['联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：',
 '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：',
 '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9534户；宽带发展1.86万户，宽带净增10625户；129元及以上实收高套发展7360户，FTTR发展4600户，奋力实现三季度收入预算完成率75%。',
 '三、营销部署（一）市场策略1.发展侧']

In [ ]:
pip install nltk

In [ ]:
pip install jieba

In [ ]:
import re
import jieba
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')  

### 3. 定义中文分词函数，定义去除停用词函数
将pdf文档中的段落做清洗，使用jieba分词提取关键词，同时去除停用词，生成关键字字符串序列（使用空格连接），供elasticSearch全文关键字检索策略使用。

In [44]:
def to_keywords(input_string):
    """将句子转成检索关键词序列"""
    # 按搜索引擎模式分词
    # word_tokens = jieba.lcut_for_search(input_string)
    word_tokens = jieba.lcut(input_string)
    # 加载停用词表
    stop_words = set(stopwords.words('chinese'))
    # 去除停用词
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

def sent_tokenize(input_string):
    """按标点断句"""
    # 按标点切分
    sentences = re.split(r'(?<=[。！？；?!])', input_string)
    # 去掉空字符串
    return [sentence for sentence in sentences if sentence.strip()]

    
if "__main__" == __name__:
    # 测试关键词提取
    # print(to_keywords("小明硕士毕业于中 国科学院计算所，后在日本京都大学深造"))
    # # 测试断句
    # print(sent_tokenize("这是，第一句。这是第二句吗？是的！啊"))
    print(to_keywords("商户市场攻坚一是健全商户队伍，现有自营厅、社区店包圈渠道聚焦周边500米，实行街长制，每条街1位负责人，支局落实所在镇区商户营销，其它区域由直销队伍转型为商客直拓队伍覆盖。二是强化行销组织，固定每周二-周三为商户营销日，挂图作战到位，定期更新作战进度；信息摸排到位，8月完成全量摸排；分级营销到位，落实营销三拜。三是加速转段升级，各区县9月底前打造1条本地标杆街，份额超30%；区县每月开展1次本地拉练赛，以赛代训沉淀经验和打法。"))
    print(sent_tokenize("商户市场攻坚一是健全商户队伍，现有自营厅、社区店包圈渠道聚焦周边500米，实行街长制，每条街1位负责人，支局落实所在镇区商户营销，其它区域由直销队伍转型为商客直拓队伍覆盖。二是强化行销组织，固定每周二-周三为商户营销日，挂图作战到位，定期更新作战进度；信息摸排到位，8月完成全量摸排；分级营销到位，落实营销三拜。三是加速转段升级，各区县9月底前打造1条本地标杆街，份额超30%；区县每月开展1次本地拉练赛，以赛代训沉淀经验和打法。"))

商户 市场 攻坚 一是 健全 商户 队伍 ， 现有 自营 厅 、 社区 店 包圈 渠道 聚焦 周边 500 米 ， 实行 街长制 ， 每条 街 1 位 负责人 ， 支局 落实 所在 镇区 商户 营销 ， 区域 直销 队伍 转型 商客 直拓 队伍 覆盖 。 二是 强化 行销 组织 ， 固定 每周 二 - 周三 商户 营销 日 ， 挂图 作战 到位 ， 定期 更新 作战 进度 ； 信息 摸排 到位 ， 8 月 全量 摸排 ； 分级 营销 到位 ， 落实 营销 三拜 。 三是 加速 转段 升级 ， 区县 9 月底 前 打造 1 条 本地 标杆 街 ， 份额 超 30% ； 区县 每月 1 次 本地 拉练 赛 ， 以赛 代训 沉淀 经验 打法 。
['商户市场攻坚一是健全商户队伍，现有自营厅、社区店包圈渠道聚焦周边500米，实行街长制，每条街1位负责人，支局落实所在镇区商户营销，其它区域由直销队伍转型为商客直拓队伍覆盖。', '二是强化行销组织，固定每周二-周三为商户营销日，挂图作战到位，定期更新作战进度；', '信息摸排到位，8月完成全量摸排；', '分级营销到位，落实营销三拜。', '三是加速转段升级，各区县9月底前打造1条本地标杆街，份额超30%；', '区县每月开展1次本地拉练赛，以赛代训沉淀经验和打法。']


### 4. 封装prompt模板

In [56]:

prompt_template = """
你是一个问答机器人。你的任务是根据以下给定的信息回答用户的提问：

已知信息：
{context}

用户的提问:
{query}
"""

def build_prompt(prompt_template,**kw):
    # **kw将传入的关键字参数放在字典中dict
    input = {}
    # kw.items()将传入的参数转成[('a',1),('b',2)]
    for key,value in kw.items():
        # 如果参数是list类型，且list的每个成员都是字符串，则在input中记录key，value键值对，value的值是用\n\n将list成员拼接起来
        if isinstance(value,list) and all([(isinstance(v,str)) for v in value]):
            input[key] = '\n\n'.join(value)
        # 否则，直接记录键值对
        else:
            input[key] = value
    return prompt_template.format(**input)

result = build_prompt(prompt_template,context="张三",query=['问题1','问题2','问题3'])
print(result)


你是一个问答机器人。你的任务是根据以下给定的信息回答用户的提问：

已知信息：
张三

用户的提问:
问题1

问题2

问题3



### 5. 封装LLM接口

In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [57]:
from openai import OpenAI
import os
# 加载环境变量
from dotenv import load_dotenv, find_dotenv

file_directory = ".env"

# print(os.getcwd())
# 获取当前目录
current_directory = os.getcwd()
# 获取当前目录的上上层目录
grandparent_directory = os.path.abspath(os.path.join(current_directory, "..\\.."))
env_path = os.path.join(grandparent_directory, file_directory)


_ = load_dotenv(find_dotenv(env_path))  # 读取本地 .env 文件，里面定义了 OPENAI_API_KEY

# openai_api_key = os.getenv('OPENAI_API_KEY')
# openai_base_url = os.getenv('OPENAI_BASE_URL')

# client = OpenAI(api_key=openai_api_key, base_url=openai_base_url)

# 新方法，不用显式传入API key和base_url
client = OpenAI()

def get_completion(prompt, model="gpt-4o"):
    '''封装 openai 接口'''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,  # 模型输出的随机性，0 表示随机性最小
    )
    return response.choices[0].message.content


get_completion("讲个笑话")

'当然！这是一个经典的笑话：\n\n有一天，小明问爸爸：“爸爸，长颈鹿的脖子为什么这么长啊？” \n\n爸爸想了想说道：“因为头离身体太远了。”'

### 5. 向量相似度计算

In [45]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    '''余弦距离 -- 越大越相似, 越接近1越相似'''
    return dot(a, b)/(norm(a)*norm(b))


def l2(a, b):
    '''欧氏距离 -- 越小越相似，越接近0越相似'''
    x = np.asarray(a)-np.asarray(b)
    return norm(x)



### 6. 封装向量化函数


In [58]:
client = OpenAI()

def get_embeddings(text,model="text-embedding-3-small",dimensions=None):
    if dimensions is None:
        data = client.embeddings.create(input=text,model=model).data
    else:
        data = client.embeddings.create(input=text,model=model,dimensitions=dimensions).data
    return [record.embedding for record in data]

test_query = ["测试文本","abc"]
test_embeddings = get_embeddings(test_query)
print(len(test_embeddings))
print(len(test_embeddings[0]))
print(test_embeddings)

2
1536
[[0.005977225489914417, 0.019384583458304405, 0.01978452317416668, -0.04906761646270752, 0.005933482199907303, -0.05259208381175995, -0.02664599008858204, -0.01296054944396019, 0.0021699860226362944, -0.021284297108650208, 0.017872311174869537, 0.0028495711740106344, -0.017359888181090355, 0.016572505235671997, 0.020209459587931633, 0.06129077821969986, -0.00763635104522109, -0.009836019948124886, -0.021471768617630005, 0.030445419251918793, 0.024371331557631493, 0.01296054944396019, 0.010142223909497261, -0.03459479287266731, -0.012210662476718426, -0.03067038394510746, -0.03831923380494118, -0.03192019462585449, 0.06379040330648422, -0.05784129723906517, -0.034294839948415756, -0.041818708181381226, 0.018697185441851616, -0.01636003889143467, -0.004136877600103617, 0.03934407979249954, 0.04609306529164314, -0.03929408639669418, 0.010417181998491287, -0.08423732221126556, -0.0225841011852026, 0.0063740406185388565, 0.02374642714858055, 0.014547810889780521, 0.02327149733901024,

In [47]:
query='国际争端'
documents=[
    "联合国就苏丹达尔富尔地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本岐阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验"
]

query_vec = get_embeddings(query)[0]
doc_vecs_list = get_embeddings(documents)
print(f'query_vec 和自身的余弦距离{cos_sim(query_vec,query_vec)}')
print('query_vec 和 doc_vecs_list 的相似度:')
for i in doc_vecs_list:
    print(cos_sim(query_vec,i))

print()

print(f'query_vec 和自身的欧式距离{l2(query_vec,query_vec)}')
print('query_vec 和 doc_vecs_list 的相似度:')
for i in doc_vecs_list: 
    print(l2(query_vec,i))

query_vec 和自身的余弦距离1.0
query_vec 和 doc_vecs_list 的相似度:
0.3456496780129385
0.28174757051137067
0.19817267934082308
0.18751817834041917
0.17089020830345558

query_vec 和自身的欧式距离0.0
query_vec 和 doc_vecs_list 的相似度:
1.1439845331208756
1.1985428109824365
1.2663548914840785
1.274740601037181
1.2877187758375583


### 7. 封装向量数据库的操作函数

In [ ]:
pip install --upgrade pip

In [ ]:
pip install chromadb -v

In [ ]:
import chromadb
from chromadb.config import Settings

# chroma_client = chromadb.Client(Settings(allow_reset=True))


# chroma_client = chromadb.EphemeralClient()

class vector_db_handler:
    def __init__(self,collection_name,embedding_fn):
        db_client = chromadb.EphemeralClient(Settings(allow_reset=True))
        db_client.reset()
        self.collection = db_client.get_or_create_collection(name=collection_name)
        self.embedding_fn = embedding_fn

    def add_documents(self,documents):
        self.collection.add(
            documents=documents,
            embeddings=self.embedding_fn(documents),
            # metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
            ids=[f"id{i}" for i in range(len(documents))]
        )
    def query_documents(self,query_string,n_results=1):
        results = self.collection.query(
            query_embeddings = self.embedding_fn([query_string]),
            n_results=n_results
        )
        return results
        
vector_db = vector_db_handler('demo',get_embeddings)
vector_db.add_documents(['（5）为强化异网客户获取，进一步释放靓号资源，AAA（非6/8/9）级靓号预存调整为500元，不带4号码保底调整为129元，带4号码保底调整为100元。',
 '（6）融合加装第二条宽带首年半价体验政策延期。（7）为落实省分公司划小积分管理要求，自8月起对办理U享小金融模式一的员工积分核算规则进行优化，由原T+1月手工核算调整为系统自动核算（分3个月核算月包积分+宽带组网积分，以39.9元FTTR小金融为例，39.9元分3个月出120积分+120元FTTR积分，总分240分不变）。',
 '2.存量侧重点落实流量经营攻坚、融合化攻坚两项工作。（1）流量运营攻坚：重点攻坚底部低活跃客户重点客群，开展流量促活攻坚。流量促活使用省公司统一产品及激励，无门槛“促活、保活”产品按2分（元）/户标准核发受理积分/激励，对于促活成功（促活后流量>1GB）次月、次次月（回落至1GB及以下不予核发），按5分（元）/月核发促活积分/激励。',
 '（2）融合化攻坚：针对入网6个月及以上存量名单制用户，新增“宽带月包”产品启动专项迁转，稳定用户在网。',
 '（3）新增产品及策略、组织要求见附件2。（4）三季度大流量包政策参照二季度继续延期。（二）营销组织',
 '1.城区市场攻坚一是全面推进以店包片，三季度全市区县分四批完成责任田划分、1+2责任人落位。二是刚性落实包区动作，包区自控社区店参照自营厅模式开展标准化运营。三是产能提升责任落实，三'])
results = vector_db.query_documents("靓号政策")

for i in results['documents'][0]:
    print(i + '\n')


（5）为强化异网客户获取，进一步释放靓号资源，AAA（非6/8/9）级靓号预存调整为500元，不带4号码保底调整为129元，带4号码保底调整为100元。



### 8. 封装RAG类

In [85]:
vector_db = vector_db_handler('demo',get_embeddings)
vector_db.add_documents(pdf_segment(text_line,min_line_length=10))

class RAGchat:
  def __init__(self,vector_db,llm_api,n_result=3):
    self.vector_db = vector_db
    self.llm_api = llm_api
    self.n_result = n_result
  def chat(self,query_text):
    result = self.vector_db.query_documents(query_text,self.n_result)
    prompt = build_prompt(prompt_template,context=result['documents'][0],query=[query_text])
    return self.llm_api(prompt)
    

rag_instance = RAGchat(vector_db,get_completion,3)

query = "三季度营销活动的时间是什么？"

result = rag_instance.chat(query)
print(result)

['联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：', '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：', '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9534户；宽带发展1.86万户，宽带净增10625户；129元及以上实收高套发展7360户，FTTR发展4600户，奋力实现三季度收入预算完成率75%。', '三、营销部署（一）市场策略1.发展侧', '联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：', '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：', '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9534户；宽带发展1.86万户，宽带净增10625户；129元及以上实收高套发展7360户，FTTR发展4600户，奋力实现三季度收入预算完成率75%。', '三、营销部署（一）市场策略1.发展侧', '联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：', '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：', '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9534户；宽带发展1.86万户，宽带净增10625户；129元及以上实收高套发展7360户，FTTR发展4600户，奋力实现三季度收入预算完成率75%。', '三、营销部署（一）市场策略1.发展侧', '联通宜昌营销字〔2024〕31号 关于开展公众市场三季度营销活动的通知各区县分公司：', '三季度是全年承上启下的关键节点，锁定三季度主营收入增长、用户规模净增、发展结构改善的主目标,宜昌分公司决定组织开展公众市场三季度营销活动，具体内容如下：', '一、活动时间2024年7月1日-2024年9月30日二、营销目标全市移网发展7万户，移网净增9